# Network Connectivity and Components Analysis

This notebook demonstrates how to analyze network connectivity and components using NetworkX.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

# Set plot style
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Connected Components in Undirected Graphs

In [ ]:
# Create a graph with multiple components
G = nx.Graph()

# Component 1
G.add_edges_from([(1, 2), (1, 3), (2, 3), (3, 4), (4, 5)])

# Component 2
G.add_edges_from([(6, 7), (7, 8), (8, 9)])

# Component 3
G.add_edges_from([(10, 11), (11, 12)])

# Find all connected components
components = list(nx.connected_components(G))
print(f"Number of components: {len(components)}")
for i, comp in enumerate(components):
    print(f"Component {i+1} has {len(comp)} nodes: {comp}")

# Get the largest connected component (GCC)
gcc = max(components, key=len)
print(f"\nSize of GCC: {len(gcc)}")

# Visualize the components
pos = nx.spring_layout(G, seed=42)
plt.figure(figsize=(10, 8))

# Color nodes by component
colors = ['skyblue', 'lightgreen', 'salmon']
for i, comp in enumerate(components):
    nx.draw_networkx_nodes(G, pos, nodelist=list(comp), node_color=colors[i], node_size=300)

nx.draw_networkx_edges(G, pos)
nx.draw_networkx_labels(G, pos)
plt.title("Connected Components in an Undirected Graph", fontsize=16)
plt.axis('off')
plt.show()

## 2. Strongly Connected Components in Directed Graphs

In [ ]:
# Create a directed graph
D = nx.DiGraph()

# Add edges to create strongly connected components
D.add_edges_from([
    (1, 2), (2, 3), (3, 1),  # SCC 1
    (4, 5), (5, 6), (6, 4),  # SCC 2
    (3, 4),  # Connection between SCCs
    (7, 8), (8, 9), (9, 7),  # SCC 3
    (6, 7)   # Connection between SCCs
])

# Find strongly connected components
sccs = list(nx.strongly_connected_components(D))
print(f"Number of strongly connected components: {len(sccs)}")
for i, comp in enumerate(sccs):
    print(f"SCC {i+1} has {len(comp)} nodes: {comp}")

# Find weakly connected components
wccs = list(nx.weakly_connected_components(D))
print(f"\nNumber of weakly connected components: {len(wccs)}")
for i, comp in enumerate(wccs):
    print(f"WCC {i+1} has {len(comp)} nodes: {comp}")

# Visualize the strongly connected components
pos = nx.spring_layout(D, seed=42)
plt.figure(figsize=(10, 8))

# Color nodes by SCC
colors = ['skyblue', 'lightgreen', 'salmon']
for i, comp in enumerate(sccs):
    nx.draw_networkx_nodes(D, pos, nodelist=list(comp), node_color=colors[i], node_size=300)

nx.draw_networkx_edges(D, pos, arrowsize=15)
nx.draw_networkx_labels(D, pos)
plt.title("Strongly Connected Components in a Directed Graph", fontsize=16)
plt.axis('off')
plt.show()

## 3. Articulation Points (Cut Vertices)

In [ ]:
# Create a graph with articulation points
G = nx.Graph()
G.add_edges_from([
    (1, 2), (1, 3), (2, 3),  # Triangle
    (3, 4),  # Articulation point at 3
    (4, 5), (4, 6), (5, 6),  # Triangle
    (6, 7),  # Articulation point at 6
    (7, 8), (7, 9), (8, 9)  # Triangle
])

# Find articulation points
cut_vertices = list(nx.articulation_points(G))
print(f"Articulation points: {cut_vertices}")

# Visualize the graph with articulation points highlighted
pos = nx.spring_layout(G, seed=42)
plt.figure(figsize=(10, 8))

# Draw regular nodes
nx.draw_networkx_nodes(G, pos,
                      nodelist=[n for n in G.nodes() if n not in cut_vertices],
                      node_color='skyblue',
                      node_size=300)

# Draw articulation points
nx.draw_networkx_nodes(G, pos,
                      nodelist=cut_vertices,
                      node_color='red',
                      node_size=400)

nx.draw_networkx_edges(G, pos)
nx.draw_networkx_labels(G, pos)
plt.title("Articulation Points (Cut Vertices)", fontsize=16)
plt.axis('off')
plt.show()

# Demonstrate the effect of removing an articulation point
print(f"\nConnected components before removal: {nx.number_connected_components(G)}")
G_copy = G.copy()
G_copy.remove_node(cut_vertices[0])
print(f"Connected components after removing node {cut_vertices[0]}: {nx.number_connected_components(G_copy)}")

## 4. Bridges (Cut Edges)

In [ ]:
# Create a graph with bridges
G = nx.Graph()
G.add_edges_from([
    (1, 2), (1, 3), (2, 3),  # Triangle
    (3, 4),  # Bridge
    (4, 5), (4, 6), (5, 6),  # Triangle
    (6, 7),  # Bridge
    (7, 8), (7, 9), (8, 9)  # Triangle
])

# Find bridges
bridges = list(nx.bridges(G))
print(f"Bridges: {bridges}")

# Visualize the graph with bridges highlighted
pos = nx.spring_layout(G, seed=42)
plt.figure(figsize=(10, 8))

# Draw nodes
nx.draw_networkx_nodes(G, pos, node_color='skyblue', node_size=300)

# Draw regular edges
regular_edges = [e for e in G.edges() if e not in bridges and (e[1], e[0]) not in bridges]
nx.draw_networkx_edges(G, pos, edgelist=regular_edges, width=1.0)

# Draw bridges
nx.draw_networkx_edges(G, pos, edgelist=bridges, width=2.0, edge_color='red')

nx.draw_networkx_labels(G, pos)
plt.title("Bridges (Cut Edges)", fontsize=16)
plt.axis('off')
plt.show()

# Demonstrate the effect of removing a bridge
print(f"\nConnected components before removal: {nx.number_connected_components(G)}")
G_copy = G.copy()
G_copy.remove_edge(*bridges[0])
print(f"Connected components after removing edge {bridges[0]}: {nx.number_connected_components(G_copy)}")

## 5. Network Resilience Analysis

In [ ]:
# Create a scale-free network
G = nx.barabasi_albert_graph(100, 2, seed=42)
original_size = len(G.nodes())

# Analyze random vs targeted attacks
def analyze_attack(G, attack_type, steps=20):
    G_copy = G.copy()
    gcc_sizes = []
    removed_nodes = []

    for i in range(steps):
        if len(G_copy.nodes()) == 0:
            break

        # Select node to remove based on attack type
        if attack_type == 'random':
            node_to_remove = np.random.choice(list(G_copy.nodes()))
        elif attack_type == 'targeted':
            node_to_remove = max(G_copy.degree, key=lambda x: x[1])[0]

        removed_nodes.append(node_to_remove)
        G_copy.remove_node(node_to_remove)

        # Measure size of largest component
        if len(G_copy.nodes()) > 0:
            components = list(nx.connected_components(G_copy))
            if components:
                gcc_size = len(max(components, key=len))
                gcc_sizes.append(gcc_size / len(G_copy.nodes()))
            else:
                gcc_sizes.append(0)

    return gcc_sizes, removed_nodes

# Run both types of attacks
random_results, random_removed = analyze_attack(G, 'random')
targeted_results, targeted_removed = analyze_attack(G, 'targeted')

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(random_results) + 1), random_results, 'b-', label='Random Removal')
plt.plot(range(1, len(targeted_results) + 1), targeted_results, 'r-', label='Targeted Removal')
plt.xlabel('Number of Nodes Removed')
plt.ylabel('Relative Size of Giant Component')
plt.title('Network Resilience: Random vs Targeted Attacks')
plt.legend()
plt.grid(True)
plt.show()

print(f"After removing {len(random_results)} nodes randomly, the relative GCC size is {random_results[-1]:.3f}")
print(f"After removing {len(targeted_results)} high-degree nodes, the relative GCC size is {targeted_results[-1]:.3f}")

## 6. Practical Application: Transportation Network Analysis

In [ ]:
# Create a transportation network (simplified example)
G = nx.Graph()

# Add cities as nodes
cities = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
G.add_nodes_from(cities)

# Add connections between cities
connections = [
    ('A', 'B'), ('A', 'C'), ('B', 'C'), ('B', 'D'),
    ('C', 'E'), ('D', 'E'), ('D', 'F'), ('E', 'F'),
    ('F', 'G'), ('G', 'H'), ('G', 'I'), ('H', 'J'), ('I', 'J')
]
G.add_edges_from(connections)

# Find critical points in the network
articulation_points = list(nx.articulation_points(G))
bridges = list(nx.bridges(G))

print(f"Critical cities (articulation points): {articulation_points}")
print(f"Critical connections (bridges): {bridges}")

# Visualize the transportation network
pos = nx.spring_layout(G, seed=42)
plt.figure(figsize=(12, 10))

# Draw regular nodes
nx.draw_networkx_nodes(G, pos,
                      nodelist=[n for n in G.nodes() if n not in articulation_points],
                      node_color='skyblue',
                      node_size=500)

# Draw articulation points
nx.draw_networkx_nodes(G, pos,
                      nodelist=articulation_points,
                      node_color='red',
                      node_size=600)

# Draw regular edges
regular_edges = [e for e in G.edges() if e not in bridges and (e[1], e[0]) not in bridges]
nx.draw_networkx_edges(G, pos, edgelist=regular_edges, width=1.5)

# Draw bridges
nx.draw_networkx_edges(G, pos, edgelist=bridges, width=2.5, edge_color='red')

nx.draw_networkx_labels(G, pos, font_size=14, font_weight='bold')
plt.title("Transportation Network Analysis", fontsize=16)
plt.axis('off')
plt.show()

# Analyze the impact of removing a critical city
if articulation_points:
    critical_city = articulation_points[0]
    print(f"\nAnalyzing impact of removing city {critical_city}:")
    print(f"Connected components before removal: {nx.number_connected_components(G)}")

    G_copy = G.copy()
    G_copy.remove_node(critical_city)

    new_components = list(nx.connected_components(G_copy))
    print(f"Connected components after removal: {len(new_components)}")

    print("\nIsolated regions after removal:")
    for i, comp in enumerate(new_components):
        print(f"Region {i+1}: {comp}")